# Project 3

## Part 2: Modeling

Model data for fun and profit.

### 0. Imports and Preliminaries

In [44]:
# imports
import pandas as pd
import numpy as np

# preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

# models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

# metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

# cross-validation
from sklearn.model_selection import train_test_split, cross_val_score

# pipelines, gridsearch
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# nltk - for stopwords and stemming
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import word_tokenize

# custom
import ipynb_utils as ipyutils

In [2]:
# load data
df = pd.read_json('../data/scrapes-clean.json', orient='index')

# convert time to datetime object
df['time'] = pd.to_datetime(df['time'], format=ipyutils.DATE_FMT)

In [3]:
# check that all looks good...
df.head()

,time,title,body-text,title-cc,title-wc,body-cc,body-wc,media,comments
0,2022-02-01,Saturn Return MEGATHREAD - we've been getting ...,,214,35,0,0,0,330
1,2022-06-01,"MERCURY RX INFOGRAPHIC: Taurus/Gemini, Apr-Jun...",,51,8,0,0,0,22
2,2022-08-30,CHANI app issues?,I just downloaded the CHANI app to try out and...,17,3,221,40,0,4
4,2022-08-30,Is Mercury in Aquarius in the 6th House as pow...,Not new to the deeper parts of astrology but t...,86,17,314,56,0,8
5,2022-08-30,What is the proper orb for a sextile?,What is the proper and respective orb for a se...,37,7,224,42,0,8


In [4]:
# ... and that the right datatypes are showing
df.dtypes

time         datetime64[ns]
title                object
body-text            object
title-cc              int64
title-wc              int64
body-cc               int64
body-wc               int64
media                 int64
comments              int64
dtype: object

In [5]:
df.shape

(1972, 9)

### 0.5. Problem Statement

What characteristics of a post on Reddit are most predictive of the overall interaction on a thread (as measured by number of comments)?

Model will attempt to predict whether or not a given Reddit post will have above or below the median number of comments.

### 1. Generate Target

In [6]:
# median comments
median = np.median(df['comments'])
median

26.0

In [7]:
# target column
df['comments_gt_median'] = (df['comments'] > median).astype(int)
df['comments_gt_median'].value_counts()

0    993
1    979
Name: comments_gt_median, dtype: int64

In [8]:
df['comments_gt_median'].value_counts(normalize=True)

0    0.50355
1    0.49645
Name: comments_gt_median, dtype: float64

#### Baseline
Baseline is just about **50%**, as it should be since we are using median as split for determining high vs. low engagement.

### 1a. Split Time Column

Might want to check by month or day of week

In [9]:
df['day'] = df['time'].apply(ipyutils.get_day_of_week)

In [10]:
df['month'] = df['time'].apply(lambda x: x.month)

In [11]:
df[['day','month']].head()

,day,month
0,1,2
1,2,6
2,1,8
4,1,8
5,1,8


In [12]:
df['weekend'] = (df['day'] > 5).astype(int)
df['weekend'].head(10)

0     0
1     0
2     0
4     0
5     0
6     0
7     1
8     1
10    1
11    1
Name: weekend, dtype: int64

In [13]:
# Doing this just to be safe as I've gotten some weird row mismatches
# later on and not sure exactly why
df.reset_index(drop=True, inplace=True)

### 2. Train-Test Split

In [14]:
col_target = 'comments_gt_median'
cols_to_drop = ['time'] # don't need this any more
X = df.drop(columns=[col_target]+cols_to_drop)
y = df[col_target]

# split to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    test_size=0.3,
                                                    random_state=1)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1380, 11), (592, 11), (1380,), (592,))

### 3. Count Vectorize Text Fields

In [59]:
# utility functions for testing stemming - taken from course 
# materials 33-nlp-ii
def stem_tokenizer(doc):
    stemmer = PorterStemmer()
    tokens = word_tokenize(doc)
    return [stemmer.stem(t) for t in tokens]

def lemma_tokenizer(doc):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(doc)
    return [lemmatizer.lemmatize(t) for t in tokens]

# I tried these on the CountVectorizer and they result in some bogus matches
# (like whitespace and punctuation). I don't have time to really look into this,
# and the scores from my tests on these were not very different from not using
# them, so I'm not going to use them this time around.

In [77]:
# get count vectorize tables
cv_params = {
    'token_pattern': ipyutils.PAT_TOKEN,
    'min_df': 2,
    'stop_words': stopwords.words('english'),
    'ngram_range': (1,2),
    'tokenizer': None # tried stem_tokenizer, lemma_tokenizer
}
cv_title = CountVectorizer(**cv_params)
cv_body = CountVectorizer(**cv_params)
cv_alltext = CountVectorizer(**cv_params)

# title
train_title_cv = cv_title.fit_transform(X_train['title'])
test_title_cv = cv_title.transform(X_test['title'])

# body
train_body_cv = cv_body.fit_transform(X_train['body-text'])
test_body_cv = cv_body.transform(X_test['body-text'])

# title + body
train_alltext_cv = cv_alltext.fit_transform(X_train['title'] + ' ' + X_train['body-text'])
test_alltext_cv = cv_alltext.transform(X_test['title'] + ' ' + X_test['body-text'])

(train_title_cv.shape, test_title_cv.shape, 
train_body_cv.shape, test_body_cv.shape,
train_alltext_cv.shape, test_alltext_cv.shape)

((1380, 1440),
 (592, 1440),
 (1380, 6368),
 (592, 6368),
 (1380, 7440),
 (592, 7440))

In [78]:
cv_title.get_feature_names_out()[-20:]

array(['years', 'years ago', 'years day', 'yes', 'yesterday', 'yet',
       'yods', 'youtube', 'yr', 'zodiac', 'zodiac beauty',
       'zodiac collection', 'zodiac illustration', 'zodiac series',
       'zodiac sign', 'zodiac signs', 'zodiac war', 'zodiac witch',
       'zodiacal', 'zodiacs'], dtype=object)

In [79]:
cv_alltext.get_feature_names_out()[-40:]

array(['year transits', 'yearly', 'years', 'years ago', 'years day',
       'years old', 'years sign', 'years since', 'years still', 'yes',
       'yesterday', 'yet', 'yet never', 'yikes', 'yods', 'yoga', 'york',
       'young', 'young people', 'younger', 'youtube', 'youtube videos',
       'yr', 'yt', 'zealot', 'zeitgeist', 'zeus', 'zodiac',
       'zodiac beauty', 'zodiac collection', 'zodiac illustration',
       'zodiac series', 'zodiac sign', 'zodiac signs', 'zodiac war',
       'zodiac witch', 'zodiacal', 'zodiacs', 'zone', 'zoom'],
      dtype=object)

### 3. Random Forest Classifier

In [80]:
title_rfc = RandomForestClassifier()
gs_params = {
    'n_estimators': [200, 300],
    'min_samples_leaf': [4, 5],
    'min_samples_split': [4, 5],
    'min_impurity_decrease': [0.0001, 0.001],
    'n_jobs': [-1],
    'random_state': [1]
}

# use gridsearch this time only to check best model params (takes a long time)
gs = GridSearchCV(title_rfc, gs_params, verbose=1, n_jobs=-1)

In [81]:
# model on titles only
gs.fit(train_title_cv, y_train)
print()
print(ipyutils.score_report(gs, 
                            (train_title_cv, y_train), 
                            (test_title_cv, y_test)))

Fitting 5 folds for each of 16 candidates, totalling 80 fits

Model Train Score (best): 0.722463768115942
Model Test Score (best): 0.6148648648648649
Model Best Estimator: RandomForestClassifier(min_impurity_decrease=0.0001, min_samples_leaf=4,
                       min_samples_split=4, n_estimators=200, n_jobs=-1,
                       random_state=1)



In [82]:
gs.best_params_

{'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 200,
 'n_jobs': -1,
 'random_state': 1}

In [83]:
# save best params to use for later models
rfc_params = gs.best_params_
# {
#     'min_impurity_decrease': 0.0001,
#     'min_samples_leaf': 5,
#     'min_samples_split': 4,
#     'n_estimators': 300,
#     'n_jobs': -1,
#     'random_state': 1
# }

In [84]:
# model on body text only - use same best params from gridsearch
body_rfc = RandomForestClassifier(**rfc_params)
body_rfc.fit(train_body_cv, y_train)
print()
print(ipyutils.score_report(body_rfc, 
                            (train_body_cv, y_train), 
                            (test_body_cv, y_test)))


Model Train Score (best): 0.7615942028985507
Model Test Score (best): 0.6537162162162162



In [85]:
# model on all text
alltext_rfc = RandomForestClassifier(**rfc_params)
alltext_rfc.fit(train_alltext_cv, y_train)
print()
print(ipyutils.score_report(alltext_rfc, 
                            (train_alltext_cv, y_train), 
                            (test_alltext_cv, y_test)))


Model Train Score (best): 0.8289855072463768
Model Test Score (best): 0.6638513513513513



#### Metrics

In [86]:
# title words
print(ipyutils.metrics_report(gs.best_estimator_, y_test, test_title_cv))

              precision    recall  f1-score   support

        high       0.60      0.71      0.65       298
         low       0.64      0.52      0.57       294

    accuracy                           0.61       592
   macro avg       0.62      0.61      0.61       592
weighted avg       0.62      0.61      0.61       592

True Positives: 153
True Negatives: 211
False Positives: 87
False Negatives: 141



In [87]:
# body words
print(ipyutils.metrics_report(body_rfc, y_test, test_body_cv))

              precision    recall  f1-score   support

        high       0.71      0.53      0.61       298
         low       0.62      0.78      0.69       294

    accuracy                           0.65       592
   macro avg       0.66      0.65      0.65       592
weighted avg       0.66      0.65      0.65       592

True Positives: 229
True Negatives: 158
False Positives: 140
False Negatives: 65



In [88]:
# all words
print(ipyutils.metrics_report(alltext_rfc, y_test, test_alltext_cv))

              precision    recall  f1-score   support

        high       0.64      0.74      0.69       298
         low       0.69      0.58      0.63       294

    accuracy                           0.66       592
   macro avg       0.67      0.66      0.66       592
weighted avg       0.67      0.66      0.66       592

True Positives: 171
True Negatives: 222
False Positives: 76
False Negatives: 123



#### Analysis of Random Forest Classifier Score

Perhaps unsurprisingly, analyzing on the full text (body plus title) gave better prediction accuracy. However, for purposes of the problem statement, the title and body are possibly best kept separate, as reddit does not diplay the full body text by default, and searches only display titles.

Accuracy is better than baseline, but not by a huge amount. Gridsearch does not reveal too much about the possible model parameters - it just tells me that the more specific model scores better.

The model is overfit (which is probably to be expected from a decision-tree-based model).

#### Exploration of Model Results

In [89]:
# title exploration - what words were best predictors?
# get predictions
preds_test = gs.best_estimator_.predict(test_title_cv)
Xdf = pd.DataFrame(test_title_cv.A, 
                   columns=cv_title.get_feature_names_out(),
                   index=y_test.index)
# get filter for correct predictions (use index to label)
preds_correct_filt = (preds_test == y_test)

In [90]:
# get word counts for correct and incorrect predictions
wordcounts = pd.DataFrame()
wordcounts['correct'] = Xdf.loc[preds_correct_filt].sum()
wordcounts['incorrect'] = Xdf[~preds_correct_filt].sum()

# get difference of correct and incorrect predictions per word
# This shows what words had more correct over incorrect predictions
wordcounts['diff'] = wordcounts['correct'] - wordcounts['incorrect']

wordcounts.sort_values(by='diff', ascending=False).head(10)

,correct,incorrect,diff
moon,40,22,18
aquarius,16,2,14
planet,18,4,14
chart,40,26,14
scorpio,15,3,12
planets,22,10,12
anyone,14,3,11
book,14,4,10
astrology,47,38,9
pluto,17,8,9


### 3a. ExtraTrees Classifier

In [91]:
title_etc = ExtraTreesClassifier(n_jobs=-1, random_state=1)
# use same gs_params from random forest
title_etc_gs = GridSearchCV(title_etc, gs_params, verbose=1, n_jobs=-1)
title_etc_gs.fit(train_title_cv, y_train)
print(ipyutils.score_report(title_etc_gs,
                            (train_title_cv, y_train),
                            (test_title_cv, y_test)))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Model Train Score (best): 0.7702898550724637
Model Test Score (best): 0.6233108108108109
Model Best Estimator: ExtraTreesClassifier(min_impurity_decrease=0.0001, min_samples_leaf=4,
                     min_samples_split=4, n_estimators=200, n_jobs=-1,
                     random_state=1)



In [92]:
etc_params = {
    'min_impurity_decrease': 0.0001,
    'min_samples_leaf': 4,
    'min_samples_split': 4,
    'n_estimators': 200,
    'n_jobs': -1,
    'random_state': 1
}

In [93]:
body_etc = ExtraTreesClassifier(**etc_params)
body_etc.fit(train_body_cv, y_train)
print(ipyutils.score_report(body_etc,
                            (train_body_cv, y_train),
                            (test_body_cv, y_test)))

Model Train Score (best): 0.7934782608695652
Model Test Score (best): 0.6672297297297297



In [94]:
alltext_etc = ExtraTreesClassifier(**etc_params)
alltext_etc.fit(train_alltext_cv, y_train)
print(ipyutils.score_report(alltext_etc,
                            (train_alltext_cv, y_train),
                            (test_alltext_cv, y_test)))

Model Train Score (best): 0.8695652173913043
Model Test Score (best): 0.6722972972972973



#### Analysis of Extra Trees Classifier Score

ExtraTrees did not fare any better than Random Forest on this data.

### 4. Other Classifiers (Quick Comparisons)

I am testing a number of other classifiers on the alltext set to see how they compare.

In [32]:
# Ada Boost
rfc = RandomForestClassifier(**rfc_params)
ada = AdaBoostClassifier(rfc, random_state=1)
ada.fit(train_alltext_cv, y_train)
ada.score(train_alltext_cv, y_train), ada.score(test_alltext_cv, y_test)

(0.9884057971014493, 0.6638513513513513)

In [33]:
# K Neighbors
knc = KNeighborsClassifier(5)
knc.fit(train_alltext_cv, y_train)
knc.score(train_alltext_cv, y_train), knc.score(test_alltext_cv, y_test)

(0.6659420289855073, 0.5168918918918919)

In [34]:
# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(train_alltext_cv, y_train)
knc.score(train_alltext_cv, y_train), knc.score(test_alltext_cv, y_test)

(0.6659420289855073, 0.5168918918918919)

In [35]:
# Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(train_alltext_cv, y_train)
mnb.score(train_alltext_cv, y_train), mnb.score(test_alltext_cv, y_test)

(0.8485507246376811, 0.6706081081081081)

#### Analysis of Other Classifiers on Word Vectors

Naive Bayes performed best, with AdaBoost second. AdaBoost was severely overfit. Naive Bayes was a bit more balanced.

### 4a. Other Features with Various Classifiers

There are a few other features I'd like to explore (word/character counts, for example).

Date/time features might not be appropriate here due to how Reddit works and the scraping process. Reddit no longer allows search by date, so I cannot get consecutive posts over time, and I am therefore trying to get as many posts I can via searches for words. Therefore, the post distribution over time that I get from my scrapes may not be the same as the actual post distribution over time, and there is no way to verify this with my current scraping process. If I have enough posts from different dates I could possibly consider distribution requirement satisfied, but I'm a bit skeptical now.

In [36]:
df.columns

Index(['time', 'title', 'body-text', 'title-cc', 'title-wc', 'body-cc',
       'body-wc', 'media', 'comments', 'comments_gt_median', 'day', 'month',
       'weekend'],
      dtype='object')

#### Word- and Character-counts and Media Indicator

In [37]:
# I'm going to test all of the following feature combinations just to see if
# they show any major differences
col_opts = [
    ['media', 'title-cc', 'body-cc', 'title-wc', 'body-wc'],
    ['media'],
    ['title-cc', 'title-wc'],
    ['body-cc', 'body-wc'],
    ['title-cc', 'title-wc', 'body-cc', 'body-wc'],
    ['day'],
    ['month'],
    ['day', 'month'],
    ['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day']
]

In [38]:
# loop over each feature combination and run a model
for opt in col_opts:
    xrfc = RandomForestClassifier(**rfc_params)
    xrfc.fit(X_train[opt], y_train)
    print(f'{opt}\n\ttrain: {xrfc.score(X_train[opt], y_train)}'\
          + f'\n\ttest: {xrfc.score(X_test[opt], y_test)}')

['media', 'title-cc', 'body-cc', 'title-wc', 'body-wc']
	train: 0.8028985507246377
	test: 0.6351351351351351
['media']
	train: 0.5434782608695652
	test: 0.543918918918919
['title-cc', 'title-wc']
	train: 0.6659420289855073
	test: 0.5337837837837838
['body-cc', 'body-wc']
	train: 0.6855072463768116
	test: 0.5929054054054054
['title-cc', 'title-wc', 'body-cc', 'body-wc']
	train: 0.8079710144927537
	test: 0.6131756756756757
['day']
	train: 0.6181159420289855
	test: 0.597972972972973
['month']
	train: 0.6876811594202898
	test: 0.7010135135135135
['day', 'month']
	train: 0.6876811594202898
	test: 0.7010135135135135
['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day']
	train: 0.8028985507246377
	test: 0.6452702702702703


In [39]:
# now let's do the same but adding the Vectorized title

# make vectorized title dataframes
train_title_cv_df = ipyutils.df_from_cv(cv_title, train_title_cv, X_train.index)
test_title_cv_df = ipyutils.df_from_cv(cv_title, test_title_cv, X_test.index)

# concat all column combos to vectorized title dataframes
combo_dfs = list()
for opt in col_opts:
    # 0 is train, 1 is test
    combo_dfs.append((ipyutils.easy_concat(X_train[opt], train_title_cv_df),
                      ipyutils.easy_concat(X_test[opt], test_title_cv_df)))

# check for integrity
for cdf in combo_dfs:
    print(cdf[0].shape, cdf[1].shape)

(1380, 1445) (592, 1445)
(1380, 1441) (592, 1441)
(1380, 1442) (592, 1442)
(1380, 1442) (592, 1442)
(1380, 1444) (592, 1444)
(1380, 1441) (592, 1441)
(1380, 1441) (592, 1441)
(1380, 1442) (592, 1442)
(1380, 1445) (592, 1445)


In [40]:
# run a model on each combo
for ix, cdf in enumerate(combo_dfs):
    opt = col_opts[ix]
    xrfc.fit(cdf[0], y_train)
    print(f'{opt}\n\ttrain:{xrfc.score(cdf[0], y_train)}\n'\
          + f'\ttest:{xrfc.score(cdf[1], y_test)}')

['media', 'title-cc', 'body-cc', 'title-wc', 'body-wc']
	train:0.7028985507246377
	test:0.643581081081081
['media']
	train:0.6992753623188406
	test:0.6131756756756757
['title-cc', 'title-wc']
	train:0.6905797101449276
	test:0.6452702702702703
['body-cc', 'body-wc']
	train:0.7057971014492753
	test:0.6351351351351351
['title-cc', 'title-wc', 'body-cc', 'body-wc']
	train:0.7043478260869566
	test:0.6469594594594594
['day']
	train:0.7057971014492753
	test:0.6402027027027027
['month']
	train:0.7231884057971014
	test:0.6976351351351351
['day', 'month']
	train:0.7188405797101449
	test:0.6942567567567568
['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day']
	train:0.7028985507246377
	test:0.6672297297297297


#### Analysis

From what I've seen so far, adding vectorized text data seems to even out the model a bit, with less overfitting than when using just word- and character-count features.

**However...** no model seems to be able to reach 70%+ accuracy. We are stuck in the 60% zone.

I'm doing all of this right now on a very small subset of data so all subject to change.

#### Other Stuff

In [41]:
df.groupby('day')['comments'].describe()

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
0,484.0,43.442149,40.767618,0.0,14.75,29.0,61.00,255.0
1,42.0,47.452381,75.077262,0.0,7.25,20.0,53.75,359.0
2,164.0,28.829268,38.206191,0.0,6.00,14.0,31.25,224.0
3,564.0,80.189716,175.136134,0.0,19.00,42.0,89.00,3100.0
4,307.0,35.684039,48.040096,0.0,10.00,21.0,43.00,368.0
5,199.0,27.331658,37.591512,0.0,4.50,13.0,34.00,207.0
6,212.0,52.485849,70.056475,0.0,8.00,30.0,72.25,596.0


**NOTES** Thursday seems to be a hot day for comments, with a much higher mean, median, and maximum.

In [42]:
df.groupby('month')['comments'].describe()

,count,mean,std,min,25%,50%,75%,max
month,,,,,,,,
1,9.0,97.888889,49.936070,26.0,64.00,80.0,139.00,166.0
2,8.0,159.250000,86.289794,47.0,118.75,141.0,182.00,330.0
3,108.0,23.722222,32.812362,0.0,6.00,11.5,24.25,198.0
4,159.0,24.981132,33.494468,0.0,4.00,12.0,31.50,196.0
5,174.0,29.885057,33.677466,0.0,7.25,15.5,37.00,159.0
6,124.0,26.008065,29.621322,1.0,7.00,15.0,30.25,152.0
7,254.0,34.818898,56.249292,0.0,6.00,14.0,37.50,390.0
8,188.0,26.675532,42.299619,0.0,5.00,11.0,29.25,359.0
9,927.0,73.365696,140.831429,2.0,23.00,44.0,85.00,3100.0


**NOTES** not enough month data right now, with very low counts